# Extending Bionty

Bionty is designed to be extendable in various ways. This guide covers how to:

1. Extend existing ontologies such as adding a new cell type to a cell type ontology
2. Adding new ontologies such as a new disease ontology
3. Implementing new entities that Bionty does not cover

## New terms

If new terms need to be added to an existing ontology we recommend using the complete Lamin platform including [lamindb](https://lamin.ai/docs/db/). A work in progress tutorial can be found [here](https://lamin.ai/docs/lnschema-bionty/guide/tables).

For use-cases where Bionty is run standalone, we kindly ask users to be patience since support for this use-case is currently work in progress.

## New ontologies

The easiest way to add new ontologies or versions to existing entities is to adapt the `local.yml` file in the `$home/.lamin/bionty/versions` directory.

For example, to add a new disease ontology (termed "mydiseases") with an associated version and URL, one adds the following lines to the `local.yml`.

```yaml
Disease:
  doid:
    versions:
      2023-01-30: http://purl.obolibrary.org/obo/doid/releases/2023-01-30/doid.obo
  mondo:
    versions:
      2022-10-11: http://purl.obolibrary.org/obo/mondo/releases/2022-10-11/mondo.owl
      2023-02-06: http://purl.obolibrary.org/obo/mondo/releases/2023-02-06/mondo.owl
  mydiseases:
    versions:
      2000-01-01: http://download-my-diseases.com/releases/2000-01-01/mydiseases.owl
```

If desired, the new ontology can be set as default. See [configuration](configuration.md) for more details.

## New entites

Adding new entities to Bionty requires subclassing the {class}`bionty.Entity` and modifying the `local.yml` file.

The {class}`bionty.Entity` requires several properties to be defined:

```python
id: str,
species: str,
database: str,
version: str,
filenames: Dict[str, str]
```

These are automatically populated by either the `._current.yaml` (see [configuration](configuration.md) or explicitly passed as parameters when initializing an Entity.

Hence, a new Entity MyEntity would be defined as:

```python
from bionty import Entity

class MyEntity(Entity):
    """MyEntity."""

    def __init__(
        self,
        species: str = "human",
        id: str = "ontology_id",
        database: str | None = None,
        version: str | None = None,
    ) -> None:
        super().__init__(
            id=id,
            database=database,
            version=version,
            species=species,
        )
```

The `local.yml` would then need to be extended as:


```yaml
MyEntity:
  mydatabase_1:
    versions:
      2042-01-01: http://my-url/releases/2042-01-01/mydatabase_1.owl
  mydatabase_2:
    versions:
      2042-01-01: http://my-url/releases/2042-01-01/mydatabase_2.owl
```

If the ontology file is not a standard `*.owl` file, the generation of the Pandas DataFrame might require custom code. In this case the function:

```python
    @cached_property
    def df(self) -> pd.DataFrame:
        """Pandas DataFrame."""
        self._filepath = settings.datasetdir / self.filenames.get(
            f"{self.species}_{self.database}"
        )

        if not self._filepath.exists():
            df = self._ontology_to_df(self.ontology)
            df.to_parquet(self._filepath)

        return pd.read_parquet(self._filepath).reset_index().set_index(self._id)
```

should be overwritten in the Entity class:

```python
    @cached_property
    def df(self) -> pd.DataFrame:
        """Pandas DataFrame."""
        # TODO: Implement me
        pass
        return pd.DataFrame()
```

Finally, the new Entity can be used with all Bionty functions.